In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import os

In [15]:
# get the subfolders in root folder 
folder_path = r"Resources"
sub_folder_names = os.listdir(folder_path)

In [34]:
#empty list for dataframes
df_list = []

# make dataframes for each subfolder and label each image
for sub_folder_name in sub_folder_names:
    subfolder_path = os.path.join(folder_path, sub_folder_name)
    file_names = os.listdir(subfolder_path)
    df = pd.DataFrame(file_names, columns=['image_id'])
    df['label'] = sub_folder_name
    df_list.append(df)

# concat list of dataframes into one dataframe
concat_df = pd.concat(df_list)
concat_df

,image_id,label
0,0.jpg,buildings
1,10006.jpg,buildings
2,1001.jpg,buildings
3,10014.jpg,buildings
4,10018.jpg,buildings
...,...,...
2878,9961.jpg,street
2879,9967.jpg,street
2880,9978.jpg,street
2881,9989.jpg,street


In [ ]:
# check that all image ids are unique
len(concat_df['image_id'].unique())